In [ ]:
from __future__ import print_function

import librosa.display
import librosa
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.ndimage.filters import uniform_filter1d


import dissonant
import os
import scipy
import pandas as pd
pd.options.mode.chained_assignment = None

import import_ipynb
from scipy import stats
import spleeter

from pydub import AudioSegment
import soundfile
from scipy.signal import savgol_filter
from scipy import signal
import scipy.signal
from matplotlib.lines import Line2D

In [ ]:
%run Only_features.ipynb     #defining our custom functions

In [ ]:
def feature_extraction1(y,sr, trimmed_audio_path, category = "polyphonic",  start_bpm = 80):
    #Features 

    #Tempo
    [tempo, t_tempo] = tempo_extraction(y,sr, start_bpm = start_bpm)

    #Onset
    [onset_env, onset_freq, t_onset] = onset_frequency(y,sr)

    #Loudness
    [loudness, t_loudness] = extract_loudness(y,sr)

    #pitches
    pitch_df = pitch_extraction(trimmed_audio_path,sr,category = "polyphonic")

    #Dissonance
    [dissonance, t_dissonance] = dissonance_extraction(pitch_df, sr)
    
    
    
    #Tempo & Loudness
    df_tup = list(zip(t_tempo, tempo, loudness[0]))
    df_tempo_loudness= pd.DataFrame(df_tup, columns = ["time", "tempo", "loudness"])
  
    #Pitch 
    pitch_df['pitch_score'] = pitch_df.loc[:, pitch_df.columns != 'time'].mean(axis = 1)
    pitch_df['pitch'] = pitch_df['pitch_score']

    #Merging pitch with tempo and loudness
    df_loudness_tempo_pitch = pd.merge(pitch_df[['time', 'pitch']], df_tempo_loudness, 
                                       on = "time", how = "left")

    
    #Dissonance
    df_tup = zip(t_dissonance, dissonance)
    df_dissonance= pd.DataFrame(df_tup, columns = ["time", "dissonance"])
    df_dissonance = df_dissonance.dropna()
    
    #Onset frequency
    df_tup = list(zip(t_onset, onset_freq))
    df_onset_freq= pd.DataFrame(df_tup, columns = ["time", "onset_freq"])


    ## Merging --
    #Onset frequency
    merge_onset = pd.merge(df_loudness_tempo_pitch,df_onset_freq,on='time', how='left')

    merge_onset.loc[pd.notnull(merge_onset.onset_freq)]
    
    #Dissonance
    df_all_features = pd.merge(merge_onset, df_dissonance, on = "time", how = 'left')

    df_all_features.loc[pd.notnull(df_all_features.dissonance)]
    ##Padding nas for plotting
    df_all_features = df_all_features.fillna(method='ffill')

    return df_all_features, pitch_df, onset_env